# Введение в нейронные сети

## Урок 5. Рекуррентные нейронные сети

1. Попробуйте починить сеть по словам.

2. Попробуйте изменить параметры нейронной сети, генерирующей текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший текст из получившихся и опишите предпринятые для его получения действия. Можно использовать текст другого произведения

In [1]:
import re

import numpy as np

import keras
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Input, SimpleRNN, Dense, GRU, LSTM, Bidirectional
from keras.utils import to_categorical

In [2]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') # убираем первый невидимый символ
text

'Вы — лучший ответ на проблемы, которые возникли в понедельник.\nДумайте позитивно и верьте в свою способность достигать отличных результатов.\nЕсли вы смогли в понедельник подняться с постели, значит вы супер герой.'

In [3]:
maxWordsCount = 1000
token = Tokenizer(num_words=maxWordsCount,
                  filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                  lower=True,
                  split=' ',
                  char_level=False)

In [4]:
token.fit_on_texts([text])

In [5]:
token.index_word

{1: 'вы',
 2: 'в',
 3: 'понедельник',
 4: 'лучший',
 5: 'ответ',
 6: 'на',
 7: 'проблемы',
 8: 'которые',
 9: 'возникли',
 10: 'думайте',
 11: 'позитивно',
 12: 'и',
 13: 'верьте',
 14: 'свою',
 15: 'способность',
 16: 'достигать',
 17: 'отличных',
 18: 'результатов',
 19: 'если',
 20: 'смогли',
 21: 'подняться',
 22: 'с',
 23: 'постели',
 24: 'значит',
 25: 'супер',
 26: 'герой'}

In [6]:
token.word_index

{'вы': 1,
 'в': 2,
 'понедельник': 3,
 'лучший': 4,
 'ответ': 5,
 'на': 6,
 'проблемы': 7,
 'которые': 8,
 'возникли': 9,
 'думайте': 10,
 'позитивно': 11,
 'и': 12,
 'верьте': 13,
 'свою': 14,
 'способность': 15,
 'достигать': 16,
 'отличных': 17,
 'результатов': 18,
 'если': 19,
 'смогли': 20,
 'подняться': 21,
 'с': 22,
 'постели': 23,
 'значит': 24,
 'супер': 25,
 'герой': 26}

In [7]:
dist = list(token.word_counts.items())
dist

[('вы', 3),
 ('лучший', 1),
 ('ответ', 1),
 ('на', 1),
 ('проблемы', 1),
 ('которые', 1),
 ('возникли', 1),
 ('в', 3),
 ('понедельник', 2),
 ('думайте', 1),
 ('позитивно', 1),
 ('и', 1),
 ('верьте', 1),
 ('свою', 1),
 ('способность', 1),
 ('достигать', 1),
 ('отличных', 1),
 ('результатов', 1),
 ('если', 1),
 ('смогли', 1),
 ('подняться', 1),
 ('с', 1),
 ('постели', 1),
 ('значит', 1),
 ('супер', 1),
 ('герой', 1)]

In [8]:
data = token.texts_to_sequences([text])

In [9]:
data_cat = to_categorical(data[0], num_classes=maxWordsCount)
data_cat.shape

(31, 1000)

In [10]:
inp_words = 3
n = data_cat.shape[0]-inp_words

In [11]:
X = np.array([data_cat[i:i+inp_words, :] for i in range(n)])
Y = data_cat[inp_words:]

In [12]:
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))

#model.add(GRU(128, activation='tanh'))
#model.add(LSTM(128, activation='tanh'))
#model.add(Bidirectional(SimpleRNN(128, activation='tanh')))

model.add(Dense(maxWordsCount, activation='softmax'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 128)               144512    
                                                                 
 dense (Dense)               (None, 1000)              129000    
                                                                 
Total params: 273,512
Trainable params: 273,512
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'],
              optimizer='adam')

In [15]:
history = model.fit(X, Y, batch_size=32, epochs=20)

Epoch 1/20
1/1 [==============================] - 1s 596ms/step - loss: 6.8995 - accuracy: 0.0000e+00
Epoch 2/20
1/1 [==============================] - 0s 4ms/step - loss: 6.8730 - accuracy: 0.0000e+00
Epoch 3/20
1/1 [==============================] - 0s 4ms/step - loss: 6.8462 - accuracy: 0.0714
Epoch 4/20
1/1 [==============================] - 0s 5ms/step - loss: 6.8188 - accuracy: 0.2143
Epoch 5/20
1/1 [==============================] - 0s 5ms/step - loss: 6.7905 - accuracy: 0.5000
Epoch 6/20
1/1 [==============================] - 0s 4ms/step - loss: 6.7610 - accuracy: 0.6786
Epoch 7/20
1/1 [==============================] - 0s 6ms/step - loss: 6.7300 - accuracy: 0.8571
Epoch 8/20
1/1 [==============================] - 0s 5ms/step - loss: 6.6972 - accuracy: 0.9286
Epoch 9/20
1/1 [==============================] - 0s 4ms/step - loss: 6.6621 - accuracy: 0.9286
Epoch 10/20
1/1 [==============================] - 0s 5ms/step - loss: 6.6244 - accuracy: 0.9286
Epoch 11/20
1/1 [============

In [16]:
def buildPhrase(text, str_len = 20):
    out_str = text
    in_data = token.texts_to_sequences([text])[0]
    for i in range(str_len):
        x = to_categorical(in_data[i: i+inp_words], num_classes=maxWordsCount) # преобразуем в One-Hot-encoding
        inp = x.reshape(1, inp_words, maxWordsCount)
        pred = model.predict(inp) # предсказываем OHE четвертого символа
        indx = pred.argmax(axis=1)[0]
        in_data.append(indx)
        out_str += " " + token.index_word[indx] # дописываем строку
    return out_str

In [17]:
buildPhrase("вы в понедельник")

1/1 [==============================] - 0s 14ms/step


'вы в понедельник думайте позитивно и верьте в свою способность достигать отличных результатов если вы смогли в понедельник подняться позитивно и позитивно в'